##**과제**: 수능 국어 문제 GPT-4로 풀어보기



Dataset Download: https://github.com/NomaDamas/KICE_slayer_AI_Korean

github의 data > 2023_11_KICE.json data를 colab으로 불러옵니다.

요구사항: https://www.notion.so/teamsparta/4-GPT-4-1712dc3ef514808cb792d55f5a09186e



##Import

In [1]:
import openai
import json
import requests
import numpy as np
import torch
import pandas as pd
from langchain.prompts import PromptTemplate

In [2]:
gpt_model="gpt-4o"

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

##Gather GPT-4  API Info

Colab 에서 GPT API 파일 저장하고 가져오기 ( os 환경변수, 로컬경로 등 업로드가 다 않되는 상황에서 취한 방식)

In [3]:
from google.colab import files
uploaded = files.upload()
for filename in uploaded.keys():
    print(f" {filename} is uploaded")

with open('OpenAI_APIKey.txt', 'r') as file:
    content = file.read()

Saving OpenAI_APIKey.txt to OpenAI_APIKey.txt
 OpenAI_APIKey.txt is uploaded


In [ ]:
print(content)

In [4]:
openai.api_key = content

##함수: prediction(problem)

In [6]:
#
def prediction(problem):
    completion = openai.chat.completions.create(
        messages=[
            {"role": "system", "content": "You are an assistant for solving Korean exam problems."},
            {"role": "user", "content": problem}
        ],
        model=gpt_model,
        response_format={"type": "json_object"},
    )
    return completion


##API access test

In [9]:
# GPT 출력 테스트
problem_text = "Example question: What is the correct answer for this Korean problem?  output format: json"
result = prediction(problem_text)

if result:
    print(f"GPT-4 Prediction: {result}")
else:
    print("Failed to get a prediction.")

GPT-4 Prediction: ChatCompletion(id='chatcmpl-AnbCUeMkJkU8P3v9B8FQxIjKF23GU', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{"answer": "C"}', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1736384434, model='gpt-4o-2024-08-06', object='chat.completion', service_tier=None, system_fingerprint='fp_b7d65f1a5b', usage=CompletionUsage(completion_tokens=7, prompt_tokens=39, total_tokens=46, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))


In [8]:
response_content = result.choices[0].message.content
fruit_data = json.loads(response_content)

formatted_json = json.dumps(fruit_data, indent=4, ensure_ascii=False)
print(formatted_json)

{
    "answer": "The correct answer is [insert answer here]"
}


##Data upload

In [73]:
url = 'https://raw.githubusercontent.com/NomaDamas/KICE_slayer_AI_Korean/master/data/2023_11_KICE.json'
response = requests.get(url)
data = response.json()

In [74]:
df = pd.DataFrame(data)
print(df.head())

                   id                                          paragraph  \
0    2023_11_KICE_1-3  사람들이 지속적으로 책을 읽는 이유 중 하나는 즐거움이다. 독서의 즐거움에는 여러 ...   
1    2023_11_KICE_4-9  (가)[A](중국에서 비롯된 유서(類書)는 고금의 서적에서 자료를 수집하고 항목별로...   
2  2023_11_KICE_10-13  법령의 조문은 대개 ‘A에 해당하면 B를 해야 한다.’처럼 요건과효과로 구성된 조건...   
3  2023_11_KICE_14-17  하루에 필요한 에너지의 양은 하루 동안의 총 열량 소모량인 대사량으로 구한다. 그중...   
4  2023_11_KICE_18-21  혼례를 마친 후 최척이 아내와 함께 장모를 모시고 집으로 돌아오매 하인들이 기뻐했다...   

   type                                           problems  
0     0  [{'question': '윗글의 내용과 일치하지 않는 것은?', 'choices'...  
1     0  [{'question': '(가)와 (나)에 대한 설명으로 가장 적절한 것은?', ...  
2     0  [{'question': '윗글의 내용과 일치하지 않는 것은?', 'choices'...  
3     0  [{'question': '윗글의 내용과 일치하지 않는 것은?', 'choices'...  
4     1  [{'question': '윗글에 대한 설명으로 가장 적절한 것은?', 'choic...  


In [75]:
dftest = df.copy()
print(dftest.head())

                   id                                          paragraph  \
0    2023_11_KICE_1-3  사람들이 지속적으로 책을 읽는 이유 중 하나는 즐거움이다. 독서의 즐거움에는 여러 ...   
1    2023_11_KICE_4-9  (가)[A](중국에서 비롯된 유서(類書)는 고금의 서적에서 자료를 수집하고 항목별로...   
2  2023_11_KICE_10-13  법령의 조문은 대개 ‘A에 해당하면 B를 해야 한다.’처럼 요건과효과로 구성된 조건...   
3  2023_11_KICE_14-17  하루에 필요한 에너지의 양은 하루 동안의 총 열량 소모량인 대사량으로 구한다. 그중...   
4  2023_11_KICE_18-21  혼례를 마친 후 최척이 아내와 함께 장모를 모시고 집으로 돌아오매 하인들이 기뻐했다...   

   type                                           problems  
0     0  [{'question': '윗글의 내용과 일치하지 않는 것은?', 'choices'...  
1     0  [{'question': '(가)와 (나)에 대한 설명으로 가장 적절한 것은?', ...  
2     0  [{'question': '윗글의 내용과 일치하지 않는 것은?', 'choices'...  
3     0  [{'question': '윗글의 내용과 일치하지 않는 것은?', 'choices'...  
4     1  [{'question': '윗글에 대한 설명으로 가장 적절한 것은?', 'choic...  


In [77]:
def count_probles_pro(problems):
    question_count = 0
    choices_count = 0
    answer_count = 0
    score_count = 0

    count = 0
    for problem in problems:
        count += 1
        print(f"count: {count}")
        print(problem)
        print()
        if 'question' in problem:
            question_count += 1
        if 'question' in problem:
            question = problem['question']
            print(f"question: {question}")
        if 'choices' in problem:
            choices_count += 1
        if 'score' in problem:
            score_count += 1
        if 'answer' in problem:
            answer_count += 1

    return question_count,choices_count,answer_count,score_count

question_count,choices_count,answer_count,score_count = count_probles_pro(dftest['problems'][0])
print(f"question:{question_count}, choices_count:{choices_count},answer_count: {answer_count}, score_count: {score_count}")

count: 1
{'question': '윗글의 내용과 일치하지 않는 것은?', 'choices': ['같은 책을 읽은 독자라도 서로 다른 의미를 구성할 수 있다.', '다른 독자와의 소통은 독자가 인식의 폭을 확장하도록 돕는다', '독자는 직접 경험해 보지 못했던 다양한 삶을 책의 필자를 매개로 접할 수 있다.', '독자의 배경지식, 관점, 읽기 환경, 과제는 독자의 의미 구성에 영향을 주는 독자 요인이다.', '독자는 책을 읽을 때 자신이 속한 사회나 시대의 영향을 받으며 필자와 간접적으로 대화한다'], 'answer': 4, 'score': 2}

count: 2
{'question': '다음은 학생이 독서 후 작성한 글의 일부이다. [A]를 바탕으로 ⓐ～ⓔ를 이해한 내용으로 가장 적절한 것은?', 'question_plus': "ⓐ('음악 시간에 들었던 베토벤의 교향곡 <합창>이 위대한 작품인 이유는 무엇일까?'하는 생각)에, 베토벤에 대한 책을 빌렸다. 책에서는 기약만으로 구성됐던 교향곡에 성악을 결합헤 개성을 드러냈다는 점에서 ⓑ(이 곡이 낭만주의 음악의 특징을 보여 준다고 했다.) <합창>을 해설한 부분에 이어, 베토벤의 생애에 관한 뒷부분도 읽었는데, ⓒ(이 내용들을 종합해, 절망적 상황에서도 열정적으로 자신이 좋아하는 일을 했기에 교향곡 구성의 새로움을 보여 준 명작이 탄생했음을 알게 됐다.) 이후 ⓓ(내가 진정으로 좋아하는 일이 무엇인지 나에게 묻게 되었다.) ⓔ(글 쓰는 일에서 가장 큰 행복을 느꼈던 나를 발견)할 수 있었고, 나도 어떤 상황에서든 좋아하는 일을 계속해야겠다고 생각했다.", 'choices': ['ⓐ와 ⓑ에는 모두 ‘독자 자신에게서 답을 찾는 질문’이 나타난다.', 'ⓒ와 ⓓ에는 모두 ‘책에 명시된 내용’에서 질문의 답을 찾아내는 모습이 나타난다.', 'ⓐ에는 ‘책에서 답을 찾는 질문’이, ⓔ에는 그에 대한 답을 ‘독자의 경험’에서 찾아내는 모습이 나타난다.', 'ⓑ에는 ‘책에서 답을 찾는 질문’이, ⓒ에는 그에 대한 답을 

In [67]:
len(df['problems'])

11

In [59]:
def remove_answer_and_score(problems):
    for problem in problems:
        if 'answer' in problem:
            del problem['answer']  # 移除 answer
        if 'score' in problem:
            del problem['score']  # 移除 score
    return problems

# 对 df 的 problems 列应用函数
dftest['problems'] = dftest['problems'].apply(remove_answer_and_score)

# 检查结果
print(dftest.head())

                   id                                          paragraph  \
0    2023_11_KICE_1-3  사람들이 지속적으로 책을 읽는 이유 중 하나는 즐거움이다. 독서의 즐거움에는 여러 ...   
1    2023_11_KICE_4-9  (가)[A](중국에서 비롯된 유서(類書)는 고금의 서적에서 자료를 수집하고 항목별로...   
2  2023_11_KICE_10-13  법령의 조문은 대개 ‘A에 해당하면 B를 해야 한다.’처럼 요건과효과로 구성된 조건...   
3  2023_11_KICE_14-17  하루에 필요한 에너지의 양은 하루 동안의 총 열량 소모량인 대사량으로 구한다. 그중...   
4  2023_11_KICE_18-21  혼례를 마친 후 최척이 아내와 함께 장모를 모시고 집으로 돌아오매 하인들이 기뻐했다...   

   type                                           problems  
0     0  [{'question': '윗글의 내용과 일치하지 않는 것은?', 'choices'...  
1     0  [{'question': '(가)와 (나)에 대한 설명으로 가장 적절한 것은?', ...  
2     0  [{'question': '윗글의 내용과 일치하지 않는 것은?', 'choices'...  
3     0  [{'question': '윗글의 내용과 일치하지 않는 것은?', 'choices'...  
4     1  [{'question': '윗글에 대한 설명으로 가장 적절한 것은?', 'choic...  


In [ ]:
from typing_extensions import Counter
# 加载 JSON 数据

total_score = 0
count = 0
for item in data:
    count +=1
    print(item)
    if count > 5: break

In [36]:
count = 0
for item in data:
    count +=1
    print("id: \n",item['id'])
    paragraph = item['paragraph']
    print("paragraph: \n",paragraph[:20],"...")

    for problem in item['problems']:
      question = problem['question']
      choices = problem['choices']
      print("---- question: \n",question[:20],"...")
      print("-------- choices: \n",choices[:20],"...","\n\n")
    if count > 0: break

id: 
 2023_11_KICE_1-3
paragraph: 
 사람들이 지속적으로 책을 읽는 이유  ...
---- question: 
 윗글의 내용과 일치하지 않는 것은? ...
-------- choices: 
 ['같은 책을 읽은 독자라도 서로 다른 의미를 구성할 수 있다.', '다른 독자와의 소통은 독자가 인식의 폭을 확장하도록 돕는다', '독자는 직접 경험해 보지 못했던 다양한 삶을 책의 필자를 매개로 접할 수 있다.', '독자의 배경지식, 관점, 읽기 환경, 과제는 독자의 의미 구성에 영향을 주는 독자 요인이다.', '독자는 책을 읽을 때 자신이 속한 사회나 시대의 영향을 받으며 필자와 간접적으로 대화한다'] ... 


---- question: 
 다음은 학생이 독서 후 작성한 글의  ...
-------- choices: 
 ['ⓐ와 ⓑ에는 모두 ‘독자 자신에게서 답을 찾는 질문’이 나타난다.', 'ⓒ와 ⓓ에는 모두 ‘책에 명시된 내용’에서 질문의 답을 찾아내는 모습이 나타난다.', 'ⓐ에는 ‘책에서 답을 찾는 질문’이, ⓔ에는 그에 대한 답을 ‘독자의 경험’에서 찾아내는 모습이 나타난다.', 'ⓑ에는 ‘책에서 답을 찾는 질문’이, ⓒ에는 그에 대한 답을 ‘책의 내용들을 관계 지으며’ 찾아내는 모습이 나타난다.', 'ⓓ에는 ‘독자 자신에게서 답을 찾는 질문’이, ⓔ에는 그에 대한 답을 ‘독자의 경험’에서 찾아내는 모습이 나타난다.'] ... 


---- question: 
 윗글을 읽고 ㉠에 대해 보인 반응으로 ...
-------- choices: 
 ['스스로 독서 계획을 세우고 자신에게 필요한 책을 찾아 개인적으로 읽는 과정에서 경험할 수 있겠군.', '독서 모임에서 서로 다른 관점을 확인하고 자신의 관점을 조정하는 과정에서 경험할 수 있겠군.', '개인적으로 형성한 의미를, 독서 동아리를 통해 심화하는 과정에서 경험할 수 있겠군.', '자신의 독서 경험을 담은 콘텐츠를 생산하고 공유하는 과정에서 경험할 수 있겠군.', '오프라인뿐 아

In [15]:
basic_prompt = PromptTemplate.from_template(
        """
        국어 시험 문제를 푸는 똑똑한 학생으로서 다음 문제의 답을 구하세요.
        지문을 읽고, 질문에 대한 답을 1부터 5까지의 선택지 중에 한 개만 골라서 대답해야 합니다.

        지문 :
        {paragraph}

        질문 :
        {question}

        선택지 :
        1번 - {choices_1}
        2번 - {choices_2}
        3번 - {choices_3}
        4번 - {choices_4}
        5번 - {choices_5}

        1번, 2번, 3번, 4번, 5번 중에 하나를 정답으로 고르세요.

        아래 포멧데로 결과를 주세요
        {paragraph[i]:question[i]:{choices[i],answer[i],scroe}}

        """
    )

In [26]:
print(basic_prompt)

input_variables=['choices_1', 'choices_2', 'choices_3', 'choices_4', 'choices_5', 'paragraph', 'question'] input_types={} partial_variables={} template='\n        국어 시험 문제를 푸는 똑똑한 학생으로서 다음 문제의 답을 구하세요.\n        지문을 읽고, 질문에 대한 답을 1부터 5까지의 선택지 중에 한 개만 골라서 대답해야 합니다.\n        \n        지문 : \n        {paragraph}\n        \n        질문 : \n        {question}\n        \n        선택지 :\n        1번 - {choices_1}\n        2번 - {choices_2}\n        3번 - {choices_3}\n        4번 - {choices_4}\n        5번 - {choices_5}\n        \n        1번, 2번, 3번, 4번, 5번 중에 하나를 정답으로 고르세요. 정답 :\n        '


In [ ]:
message=[{"role": "assistant", "content": basic_prompt}, {"role": "user", "content": data}]
response = client.chat.completions.create(
    model="gpt-4o",
    messages=message,
    n=1,
    max_tokens=max_tokens,
    temperature=0.0,
    frequency_penalty=frequency_penalty
)
print(response.choices[0].message.content)